In [2]:
%pip install --upgrade --quiet  langchain-pinecone langchain

Note: you may need to restart the kernel to use updated packages.


In [3]:
import os
from langchain import PromptTemplate
from langchain.chains import RetrievalQA
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import Pinecone
import pinecone
from langchain.document_loaders import PyPDFLoader, DirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.prompts import PromptTemplate
from langchain.llms import CTransformers

c:\Users\victo\Llama2&Flask\.conda\lib\site-packages\pinecone\data\index.py:1: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm


In [4]:
PINECONE_API_KEY =os.getenv("PINECONE_API_KEY")
PINECONE_API_ENV =os.getenv("PINECONE_API_ENV")

In [5]:
#Extract data from the PDF
def load_pdf(data):
    loader = DirectoryLoader(data,
                    glob="*.pdf",
                    loader_cls=PyPDFLoader)
    
    documents = loader.load()

    return documents

In [6]:
extracted_data = load_pdf("data/")

In [7]:
#Create text chunks
def text_split(extracted_data):
    text_splitter = RecursiveCharacterTextSplitter(chunk_size = 500, chunk_overlap = 20)
    text_chunks = text_splitter.split_documents(extracted_data)

    return text_chunks

In [8]:
text_chunks = text_split(extracted_data)
print("length of my chunk:", len(text_chunks))

length of my chunk: 3626


In [9]:
#download embedding model
def download_hugging_face_embeddings():
    embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
    return embeddings

In [10]:
embeddings = download_hugging_face_embeddings()

In [11]:
embeddings

HuggingFaceEmbeddings(client=SentenceTransformer(
  (0): Transformer({'max_seq_length': 256, 'do_lower_case': False}) with Transformer model: BertModel 
  (1): Pooling({'word_embedding_dimension': 384, 'pooling_mode_cls_token': False, 'pooling_mode_mean_tokens': True, 'pooling_mode_max_tokens': False, 'pooling_mode_mean_sqrt_len_tokens': False})
  (2): Normalize()
), model_name='sentence-transformers/all-MiniLM-L6-v2', cache_folder=None, model_kwargs={}, encode_kwargs={}, multi_process=False, show_progress=False)

In [12]:
query_result = embeddings.embed_query("Hello world")
print("Length", len(query_result))

Length 384


In [13]:
#Initializing the Pinecone
from pinecone import Pinecone
from langchain_pinecone import PineconeVectorStore
pc = Pinecone(api_key=PINECONE_API_KEY)
index = pc.Index("test")

index_name="test"

#Creating Embeddings for Each of The Text Chunks & storing
docsearch = PineconeVectorStore.from_documents(text_chunks, embeddings, index_name=index_name)

In [14]:
text_chunks

[Document(page_content='Chaos and Fractals', metadata={'source': 'data\\[0387202293] Chaos and Fractals.pdf', 'page': 0, 'text': 'Chaos and Fractals'}),
 Document(page_content='Springer\nNew York\nBerlin\nHeidelberg\nHong Kong\nLondon\nMilanParis\nTokyo', metadata={'source': 'data\\[0387202293] Chaos and Fractals.pdf', 'page': 1, 'text': 'Springer\nNew York\nBerlin\nHeidelberg\nHong Kong\nLondon\nMilanParis\nTokyo'}),
 Document(page_content='Springe rHeinz-Otto Peitgen Hartmut Jürgens Dietmar Saupe\nChaos and Fractals\nNew Frontiers of Science\nSecond Edition\nWith 606 illustrations, 40 in color', metadata={'source': 'data\\[0387202293] Chaos and Fractals.pdf', 'page': 2, 'text': 'Springe rHeinz-Otto Peitgen Hartmut Jürgens Dietmar Saupe\nChaos and Fractals\nNew Frontiers of Science\nSecond Edition\nWith 606 illustrations, 40 in color'}),
 Document(page_content="eBook ISBN: 0-387-21823-8\nPrint ISBN: 0-387-20229-3\n©2004 Springer Science + Business Media, Inc.\nPrint © 2004 Springer-Ve

In [15]:
query = "What are Fractals"

docs=docsearch.similarity_search(query, k=3)

print("Result", docs)

Result [Document(page_content='award-winning video  Fractals: An Animated Discussion\nDietmar Saupe. *1954 in Bremen (Germany). Dr. rer. nat. 1982 and\nHabilitation 1993, both from the University of Bremen.  Visiting As-\nsistant Professor of Mathematics at the University of California at\nSanta Cruz, 1985–87, Assistant Professor at the University of Bre-\nmen, 1987–93, Professsor of Computer Science at the University of\nFreiburg, 1993–1998, at the University of Leipzig, 1998–2002, and', metadata={'page': 9.0, 'source': 'data\\[0387202293] Chaos and Fractals.pdf'}), Document(page_content='award-winning video  Fractals: An Animated Discussion\nDietmar Saupe. *1954 in Bremen (Germany). Dr. rer. nat. 1982 and\nHabilitation 1993, both from the University of Bremen.  Visiting As-\nsistant Professor of Mathematics at the University of California at\nSanta Cruz, 1985–87, Assistant Professor at the University of Bre-\nmen, 1987–93, Professsor of Computer Science at the University of\nFreiburg

In [16]:
prompt_template="""
Use the following pieces of information to answer the user's question.
If you don't know the answer, just say that you don't know, don't try to make up an answer.

Context: {context}
Question: {question}

Only return the helpful answer below and nothing else.
Helpful answer:
"""

In [17]:
PROMPT=PromptTemplate(template=prompt_template, input_variables=["context", "question"])
chain_type_kwargs={"prompt": PROMPT}

In [18]:
llm=CTransformers(model="model/llama-2-7b-chat.ggmlv3.q4_0.bin",
                  model_type="llama",
                  config={'max_new_tokens':512,
                          'temperature':0.8})

RepositoryNotFoundError: 401 Client Error. (Request ID: Root=1-65e823e1-0cacdc13493c1b1f72a39f90;a6d5b77b-5776-401d-836c-55b4508685ef)

Repository Not Found for url: https://huggingface.co/api/models/model/llama-2-7b-chat.ggmlv3.q4_0.bin/revision/main.
Please make sure you specified the correct `repo_id` and `repo_type`.
If you are trying to access a private or gated repo, make sure you are authenticated.
Invalid username or password.

In [ ]:
qa=RetrievalQA.from_chain_type(
    llm=llm, 
    chain_type="stuff", 
    retriever=docsearch.as_retriever(search_kwargs={'k': 2}),
    return_source_documents=True, 
    chain_type_kwargs=chain_type_kwargs)

In [ ]:
while True:
    user_input=input(f"Input Prompt:")
    result=qa({"query": user_input})
    print("Response : ", result["result"])